In [1]:
import sys
sys.path.insert(0, "timm-efficientdet-pytorch")
sys.path.insert(0, "omegaconf")

In [2]:
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

### Stratify KFold

In [3]:
marking = pd.read_csv('/home/hy/dataset/gwd/train.csv')
marking

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1
...,...,...,...,...,...
147788,5e0747034,1024,1024,"[64.0, 619.0, 84.0, 95.0]",arvalis_2
147789,5e0747034,1024,1024,"[292.0, 549.0, 107.0, 82.0]",arvalis_2
147790,5e0747034,1024,1024,"[134.0, 228.0, 141.0, 71.0]",arvalis_2
147791,5e0747034,1024,1024,"[430.0, 13.0, 184.0, 79.0]",arvalis_2


In [4]:
test_bboxs = marking['bbox'].apply(lambda x: np.fromstring(x[1:-1],sep=','))

In [5]:
test_bboxs

0          [834.0, 222.0, 56.0, 36.0]
1         [226.0, 548.0, 130.0, 58.0]
2         [377.0, 504.0, 74.0, 160.0]
3         [834.0, 95.0, 109.0, 107.0]
4         [26.0, 144.0, 124.0, 117.0]
                     ...             
147788      [64.0, 619.0, 84.0, 95.0]
147789    [292.0, 549.0, 107.0, 82.0]
147790    [134.0, 228.0, 141.0, 71.0]
147791     [430.0, 13.0, 184.0, 79.0]
147792     [875.0, 740.0, 94.0, 61.0]
Name: bbox, Length: 147793, dtype: object

In [6]:
bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))

In [7]:
bboxs

array([[834., 222.,  56.,  36.],
       [226., 548., 130.,  58.],
       [377., 504.,  74., 160.],
       ...,
       [134., 228., 141.,  71.],
       [430.,  13., 184.,  79.],
       [875., 740.,  94.,  61.]])

In [8]:
for i, column in enumerate(['x', 'y', 'w', 'h']):
    marking[column] = bboxs[:,i]
marking.drop(columns=['bbox'], inplace=True)

## Marking Dataframe

In [9]:
marking

,image_id,width,height,source,x,y,w,h
0,b6ab77fd7,1024,1024,usask_1,834.0,222.0,56.0,36.0
1,b6ab77fd7,1024,1024,usask_1,226.0,548.0,130.0,58.0
2,b6ab77fd7,1024,1024,usask_1,377.0,504.0,74.0,160.0
3,b6ab77fd7,1024,1024,usask_1,834.0,95.0,109.0,107.0
4,b6ab77fd7,1024,1024,usask_1,26.0,144.0,124.0,117.0
...,...,...,...,...,...,...,...,...
147788,5e0747034,1024,1024,arvalis_2,64.0,619.0,84.0,95.0
147789,5e0747034,1024,1024,arvalis_2,292.0,549.0,107.0,82.0
147790,5e0747034,1024,1024,arvalis_2,134.0,228.0,141.0,71.0
147791,5e0747034,1024,1024,arvalis_2,430.0,13.0,184.0,79.0


In [10]:
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
df_folds = marking[['image_id']].copy()

In [11]:
df_folds

,image_id
0,b6ab77fd7
1,b6ab77fd7
2,b6ab77fd7
3,b6ab77fd7
4,b6ab77fd7
...,...
147788,5e0747034
147789,5e0747034
147790,5e0747034
147791,5e0747034


In [12]:
df_folds.loc[:,'bbox_count'] = 1; df_folds

,image_id,bbox_count
0,b6ab77fd7,1
1,b6ab77fd7,1
2,b6ab77fd7,1
3,b6ab77fd7,1
4,b6ab77fd7,1
...,...,...
147788,5e0747034,1
147789,5e0747034,1
147790,5e0747034,1
147791,5e0747034,1


In [13]:
df_folds = df_folds.groupby('image_id').count(); df_folds

,bbox_count
image_id,
00333207f,55
005b0d8bb,20
006a994f7,25
00764ad5d,41
00b5fefed,25
...,...
ffb445410,57
ffbf75e5b,52
ffbfe7cc0,34


In [14]:
df_folds.bbox_count.max()

116

In [15]:
df_folds.loc[:,'source'] = marking [['image_id','source']].groupby('image_id').min()['source']

In [16]:
marking [['image_id','source']].groupby('image_id').min()['source']

image_id
00333207f    arvalis_1
005b0d8bb      usask_1
006a994f7      inrae_1
00764ad5d      inrae_1
00b5fefed    arvalis_3
               ...    
ffb445410       rres_1
ffbf75e5b    arvalis_1
ffbfe7cc0    arvalis_1
ffc870198      usask_1
ffdf83e42    arvalis_1
Name: source, Length: 3373, dtype: object

In [17]:
df_folds

,bbox_count,source
image_id,,
00333207f,55,arvalis_1
005b0d8bb,20,usask_1
006a994f7,25,inrae_1
00764ad5d,41,inrae_1
00b5fefed,25,arvalis_3
...,...,...
ffb445410,57,rres_1
ffbf75e5b,52,arvalis_1
ffbfe7cc0,34,arvalis_1


In [18]:
df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['source'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)

In [19]:
df_folds

,bbox_count,source,stratify_group
image_id,,,
00333207f,55,arvalis_1,arvalis_1_3
005b0d8bb,20,usask_1,usask_1_1
006a994f7,25,inrae_1,inrae_1_1
00764ad5d,41,inrae_1,inrae_1_2
00b5fefed,25,arvalis_3,arvalis_3_1
...,...,...,...
ffb445410,57,rres_1,rres_1_3
ffbf75e5b,52,arvalis_1,arvalis_1_3
ffbfe7cc0,34,arvalis_1,arvalis_1_2


In [20]:
df_folds.loc[:, 'fold'] = 0;df_folds

,bbox_count,source,stratify_group,fold
image_id,,,,
00333207f,55,arvalis_1,arvalis_1_3,0
005b0d8bb,20,usask_1,usask_1_1,0
006a994f7,25,inrae_1,inrae_1_1,0
00764ad5d,41,inrae_1,inrae_1_2,0
00b5fefed,25,arvalis_3,arvalis_3_1,0
...,...,...,...,...
ffb445410,57,rres_1,rres_1_3,0
ffbf75e5b,52,arvalis_1,arvalis_1_3,0
ffbfe7cc0,34,arvalis_1,arvalis_1_2,0


In [21]:
for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

/home/hy/anaconda3/envs/badeda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [22]:
df_folds.index

Index(['00333207f', '005b0d8bb', '006a994f7', '00764ad5d', '00b5fefed',
       '00b70a919', '00e903abe', '00ea5e5ee', '010b216d4', '010c93b99',
       ...
       'ff46aa8d4', 'ff5c9104a', 'ff86547fb', 'ffaa964a2', 'ffb0f6eca',
       'ffb445410', 'ffbf75e5b', 'ffbfe7cc0', 'ffc870198', 'ffdf83e42'],
      dtype='object', name='image_id', length=3373)

In [23]:
df_folds['stratify_group']

image_id
00333207f    arvalis_1_3
005b0d8bb      usask_1_1
006a994f7      inrae_1_1
00764ad5d      inrae_1_2
00b5fefed    arvalis_3_1
                ...     
ffb445410       rres_1_3
ffbf75e5b    arvalis_1_3
ffbfe7cc0    arvalis_1_2
ffc870198      usask_1_2
ffdf83e42    arvalis_1_2
Name: stratify_group, Length: 3373, dtype: object

## KFold datafranme

In [24]:
df_folds.head()

,bbox_count,source,stratify_group,fold
image_id,,,,
00333207f,55,arvalis_1,arvalis_1_3,1
005b0d8bb,20,usask_1,usask_1_1,3
006a994f7,25,inrae_1,inrae_1_1,1
00764ad5d,41,inrae_1,inrae_1_2,0
00b5fefed,25,arvalis_3,arvalis_3_1,3


In [25]:
df_folds[df_folds['stratify_group']=='arvalis_1_2'][df_folds['fold']==3].sum()

/home/hy/anaconda3/envs/badeda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


bbox_count                                                     3629
source            arvalis_1arvalis_1arvalis_1arvalis_1arvalis_1a...
stratify_group    arvalis_1_2arvalis_1_2arvalis_1_2arvalis_1_2ar...
fold                                                            297
dtype: object

In [26]:
df_folds[df_folds['fold']==0]['bbox_count'].sum()

29369

In [27]:
df_folds[df_folds['fold']==1]['bbox_count'].sum()

29821

In [28]:
df_folds[df_folds['fold']==2]['bbox_count'].sum()

29521

In [29]:
df_folds[df_folds['fold']==3]['bbox_count'].sum()

29499

### Albumentations

In [30]:
def get_train_transforms():
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.5),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            ],p=0.9),
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1),
            A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

### Dataset

In [ ]:
TRAIN_ROOT_PATH = '/home/hy/dataset/gwd/train'

class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        
        if self.test or random.random() > 0.5:
            image, boxes = self.load_image_and_boxes(index)
        else:
            image, boxes = self.load_cutmix_image_and_boxes(index)

        # there is only one class
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]
    
    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes
    
    